In [10]:
!pip install transformers bitsandbytes accelerate huggingface_hub langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.4 MB/s eta 0:00:00


In [2]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
from torch import cuda, bfloat16
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from transformers import AutoConfig

llama2_id = 'meta-llama/Llama-2-7b-chat-hf'

model_config = AutoConfig.from_pretrained(
    llama2_id,
    use_auth_token=True,
    device_map=device
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1033: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    llama2_id,
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [7]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    llama2_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [8]:
from transformers import pipeline

pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    repetition_penalty=1.1,  # without this output begins repeating
)


In [11]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipe)

In [16]:
from langchain.evaluation import load_evaluator
from pprint import pprint as print

evaluator = load_evaluator("score_string", criteria="correctness", llm=llm)


context = """James attempted four free throws against Miami, dropping his season average to 5.7 attempts per
game, which would tie a single-season career-low should it sustain. The four-time league MVP has drawn a
foul on just 6% of his drives this season, according to Second Spectrum data, which is the eighth-lowest
rate among the 34 players to record 70 or more drives thus far in 2023-24."""

in_prompt = f"""
Contenxt: {context}

Generate a query based on the given context.
"""

pred = "How many free throws did James attempt against Miami?"

eval_result = evaluator.evaluate_strings(
    prediction=pred,
    input=in_prompt,
)

ValueError: ignored